# Realized Volatility Forecast

Predict **the realized volatility of the next 10 minutes time window** with two data sets of the last ten minutes (600 seconds). Book dataset contains ask and bid prices of almost each second, which allows us to calculate the realized volatility of the last ten minutes. Trade dataset contains the actual record of stock trading, which is more sparse.

## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import os
import glob

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

## Functions for data preprocessing

In [ ]:
def calc_wap1(df):
    wap1 = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap1

def calc_wap2(df):
    wap2 = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap2

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    return len(np.unique(series))

## Try with a single stock id

In [ ]:
book_train = pd.read_parquet(data_dir + "book_train.parquet/stock_id=15")
book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


In [ ]:
############### set the seconds for feature engineering ###############

last_seconds = [300]


## Main function for preprocessing book data

In [ ]:
# feature engineering for book data

def preprocessor_book(file_path, last_seconds):
    
    df = pd.read_parquet(file_path)
    
    #calculate return etc
    df['wap1'] = calc_wap1(df)
    df['log_return1'] = df.groupby('time_id')['wap1'].apply(log_return)
    
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['bid_ask_spread'] = df['ask_price1']/df['bid_price1'] - 1
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    #df['bid_spread'] = df['bid_price1'] / df['bid_price2']
    #df['ask_spread'] = df['ask_price1'] / df['ask_price2']

    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    #df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    df['volume_imbalance']  = (df['ask_size1'] + df['ask_size2']) / (df['bid_size1'] + df['bid_size2'])
    

    #dict for aggregate operations for the columns
    create_feature_dict = {
        'wap1':[np.mean],
        'log_return1':[realized_volatility],
        'wap2':[np.mean],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean],
        'bid_ask_spread':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'total_volume':[np.mean],
        'volume_imbalance':[np.mean],
    }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index() 
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
    
    ######groupby / last XX seconds

    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)

    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

In [ ]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path, last_seconds)

CPU times: user 8.54 s, sys: 448 ms, total: 8.99 s
Wall time: 8.86 s


,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,row_id
0,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0-5
1,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0-11
2,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0-16
3,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0-31
4,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.997938,0.002579,0.997864,0.003821,0.000212,0.000552,0.000552,0.000083,-0.000182,374.235690,9.557135,0.997519,0.001673,0.997417,0.002573,0.000193,0.000509,0.000509,0.000062,-0.000169,350.560000,15.869553,0-32751
3826,1.000310,0.002206,1.000487,0.002847,0.000267,0.000543,0.000542,0.000092,-0.000172,621.131068,14.446517,1.000682,0.001487,1.000930,0.002255,0.000300,0.000588,0.000588,0.000074,-0.000177,668.640000,20.406932,0-32753
3827,0.999552,0.002913,0.999456,0.003266,0.000237,0.000525,0.000525,0.000202,-0.000083,343.734043,2.742037,1.000111,0.001928,1.000038,0.002646,0.000216,0.000446,0.000446,0.000191,-0.000075,326.759615,2.114044,0-32758
3828,1.002357,0.003046,1.002386,0.005105,0.000245,0.000480,0.000480,0.000113,-0.000166,385.429967,3.461651,1.002277,0.002137,1.002328,0.003934,0.000269,0.000516,0.000516,0.000096,-0.000175,394.588957,5.078213,0-32763


In [ ]:
trade_train = pd.read_parquet(data_dir + "trade_train.parquet/stock_id=0")
trade_train.head(15)

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
5,5,78,1.003762,134,5
6,5,122,1.004207,102,3
7,5,127,1.004577,1,1
8,5,144,1.004370,6,1
9,5,147,1.003964,233,4


## Main function for preprocessing trade data

In [ ]:
def preprocessor_trade(file_path, last_seconds):
    
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    #######df['max_price'] = df.groupby('time_id')['price'].apply(max)
    #######df['min_price'] = df.groupby('time_id')['price'].apply(min)
    df['avg_size'] = df['size'] / df['order_count']
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique], #########
        'size':[np.sum],
        'order_count':[np.mean],
        'price': [max, min],    
        'avg_size': [np.mean]
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    ######groupby / last XX seconds

    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
   
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    #df_feature['HL_difference'] = 
    
    return df_feature

In [ ]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path, last_seconds)

CPU times: user 4.25 s, sys: 78.1 ms, total: 4.32 s
Wall time: 4.23 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,row_id
0,0.002006,40,3179,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798,0-5
1,0.000901,30,1289,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917,0-11
2,0.001961,25,2161,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111,0-16
3,0.001561,15,1962,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457,0-31
4,0.000871,22,1791,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.998751,0.996358,16.946810,0.001162,35.0,2365.0,3.257143,0.998335,0.996358,19.163118,0-32751
3826,0.001411,28,4547,3.892857,1.001227,0.999474,29.297165,0.001066,12.0,2161.0,4.250000,1.001227,1.000113,33.454861,0-32753
3827,0.001521,36,4250,3.500000,1.000705,0.998438,34.027020,0.001242,22.0,2294.0,3.727273,1.000705,0.999103,32.324518,0-32758
3828,0.001794,53,3217,2.150943,1.003003,1.001611,28.715881,0.001404,25.0,1627.0,1.920000,1.003003,1.001803,33.391667,0-32763


## Combined preprocessor function

In [ ]:
# parallel computing to save time
from joblib import Parallel, delayed 

def preprocessor(list_stock_ids, is_train = True):
    
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book, last_seconds), preprocessor_trade(file_path_trade, last_seconds), on='row_id', how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [ ]:
list_stock_ids = [0,1]
preprocessor(list_stock_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   14.8s finished


,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,row_id,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300
0,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0-5,0.002006,40,3179,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798
1,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0-11,0.000901,30,1289,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917
2,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0-16,0.001961,25,2161,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111
3,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0-31,0.001561,15,1962,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457
4,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0-62,0.000871,22,1791,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,1.000142,0.003723,1.000006,0.004996,0.000330,0.000598,0.000597,0.000157,-0.000118,296.185668,10.872423,1.000130,0.002212,1.000029,0.002954,0.000283,0.000617,0.000617,0.000165,-0.000104,322.284615,6.274222,1-32751,0.001776,49,3249,2.775510,1.000666,0.999463,16.307750,0.001280,23.0,1889.0,3.608696,1.000605,0.999682,20.530642
7656,1.007503,0.010829,1.007318,0.012168,0.000403,0.000923,0.000922,0.000159,-0.000125,567.840081,5.897374,1.012343,0.008499,1.012109,0.009971,0.000483,0.001082,0.001082,0.000196,-0.000129,485.195021,6.284400,1-32753,0.008492,183,75903,7.874317,1.016428,0.995071,52.285731,0.006310,88.0,30858.0,8.136364,1.016428,1.010031,44.155094
7657,1.000854,0.003135,1.000849,0.004268,0.000243,0.000648,0.000648,0.000141,-0.000132,426.603834,2.775426,1.001250,0.002108,1.001229,0.003184,0.000261,0.000625,0.000625,0.000146,-0.000146,426.939597,5.229650,1-32758,0.001927,26,2239,2.615385,1.001697,0.999468,43.677564,0.001567,11.0,980.0,2.727273,1.001697,1.000624,41.737879
7658,1.003032,0.003750,1.003041,0.005773,0.000199,0.000421,0.000421,0.000190,-0.000231,526.317972,2.010911,1.004296,0.002728,1.004304,0.004435,0.000195,0.000410,0.000410,0.000165,-0.000240,519.044843,2.583860,1-32763,0.002856,109,16648,2.935780,1.005453,0.999446,50.734265,0.001919,57.0,8274.0,2.701754,1.005453,1.003058,51.322710


## Training set

In [ ]:
train = pd.read_csv(data_dir + 'train.csv')

In [ ]:
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [ ]:
train_ids = train.stock_id.unique()
train_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126])

In [ ]:
%%time
df_train = preprocessor(list_stock_ids= train_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 11.0min finished


CPU times: user 3.71 s, sys: 351 ms, total: 4.06 s
Wall time: 11min


In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_train = train.merge(df_train, on = ['row_id'], how = 'left')

In [ ]:
df_train.head()

,row_id,target,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300
0,0-5,0.004136,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0.002006,40.0,3179.0,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798
1,0-11,0.001445,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0.000901,30.0,1289.0,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917
2,0-16,0.002168,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0.001961,25.0,2161.0,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111
3,0-31,0.002195,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0.001561,15.0,1962.0,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457
4,0-62,0.001747,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0.000871,22.0,1791.0,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506


## Test set

In [ ]:
test = pd.read_csv(data_dir + 'test.csv')

In [ ]:
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [ ]:
test_ids = test.stock_id.unique()

In [ ]:
%%time
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)

CPU times: user 25.1 ms, sys: 1.09 ms, total: 26.2 ms
Wall time: 196 ms


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


In [ ]:
df_test.head(20)

,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,row_id,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300
0,1.000405,0.000294,1.00055,0.000252,0.000145,0.000557,0.000557,0.000393,-0.000115,350.666667,0.455667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-4,0.000295,3,201,3.666667,1.000344,1.000049,16.206349,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

In [ ]:
df_test.head()

,stock_id,time_id,row_id,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300
0,0,4,0-4,1.000405,0.000294,1.00055,0.000252,0.000145,0.000557,0.000557,0.000393,-0.000115,350.666667,0.455667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,1.000344,1.000049,16.206349,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Target encoding by stock_id

In [ ]:
from sklearn.model_selection import KFold

#stock_id target encoding
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean() #####?????? 
print('**************************')
print('Length of stock_id_target_mean: ' ,len(stock_id_target_mean))
print()
print(stock_id_target_mean)

df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean) # test_set
print('**************************')
display(df_test)

**************************
Length of stock_id_target_mean:  112

stock_id
0      0.004028
1      0.004362
10     0.003501
100    0.004979
101    0.004082
102    0.004394
103    0.005232
104    0.003333
105    0.002635
107    0.003193
108    0.003046
109    0.004109
11     0.004609
110    0.004383
111    0.002535
112    0.005462
113    0.004190
114    0.004358
115    0.002882
116    0.004676
118    0.004743
119    0.003217
120    0.002808
122    0.003762
123    0.002506
124    0.003691
125    0.002016
126    0.005321
13     0.002489
14     0.002697
15     0.003694
16     0.004146
17     0.004137
18     0.008707
19     0.002913
2      0.002385
20     0.002910
21     0.002495
22     0.004759
23     0.004421
26     0.003163
27     0.006738
28     0.002898
29     0.001836
3      0.006182
30     0.003912
31     0.003598
32     0.002701
33     0.005860
34     0.002593
35     0.003113
36     0.003383
37     0.006660
38     0.004617
39     0.002725
4      0.004197
40     0.005652
41     0.00210

,stock_id,time_id,row_id,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id_target_enc
0,0,4,0-4,1.000405,0.000294,1.00055,0.000252,0.000145,0.000557,0.000557,0.000393,-0.000115,350.666667,0.455667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,1.000344,1.000049,16.206349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028


In [ ]:
#
tmp = np.repeat(np.nan, df_train.shape[0]) # create an empty numpy array

kf = KFold(n_splits=10, shuffle=True, random_state=2046)

for idx_1, idx_2 in kf.split(df_train): # loop the train & validation indices????
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    print('*********************')
    print(target_mean)
    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
    print('*********************')
    print(tmp)
    
df_train['stock_id_target_enc'] = tmp

*********************
stock_id
0      0.004050
1      0.004356
10     0.003502
100    0.004972
101    0.004086
102    0.004415
103    0.005239
104    0.003316
105    0.002656
107    0.003209
108    0.003051
109    0.004085
11     0.004645
110    0.004388
111    0.002546
112    0.005444
113    0.004185
114    0.004387
115    0.002899
116    0.004696
118    0.004758
119    0.003207
120    0.002810
122    0.003758
123    0.002508
124    0.003709
125    0.002013
126    0.005330
13     0.002484
14     0.002684
15     0.003696
16     0.004146
17     0.004145
18     0.008665
19     0.002919
2      0.002389
20     0.002905
21     0.002497
22     0.004792
23     0.004410
26     0.003179
27     0.006745
28     0.002908
29     0.001846
3      0.006215
30     0.003925
31     0.003599
32     0.002701
33     0.005871
34     0.002561
35     0.003133
36     0.003381
37     0.006671
38     0.004631
39     0.002718
4      0.004191
40     0.005668
41     0.002109
42     0.003251
43     0.001471
44     0.

In [ ]:
df_train.head(30)

,row_id,target,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id,stock_id_target_enc
0,0-5,0.004136,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0.002006,40.0,3179.0,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798,0,0.004025
1,0-11,0.001445,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0.000901,30.0,1289.0,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917,0,0.004057
2,0-16,0.002168,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0.001961,25.0,2161.0,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111,0,0.004050
3,0-31,0.002195,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0.001561,15.0,1962.0,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457,0,0.004025
4,0-62,0.001747,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0.000871,22.0,1791.0,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506,0,0.004025
5,0-72,0.004912,0.998605,0.007902,0.998466,0.010336,0.000649,0.001639,0.001637,0.000114,-0.000241,352.574144,2.191018,0.997851,0.006357,0.997820,0.006505,0.000683,0.001795,0.001793,0.000080,-0.000260,327.571429,1.166757,0.003443,26.0,3395.0,3.730769,0.999817,0.996360,28.982180,0.002419,14.0,1551.0,2.785714,0.997933,0.996360,30.504082,0,0.004025
6,0-97,0.009388,0.996629,0.010034,0.996725,0.014493,0.000666,0.001666,0.001665,0.000266,-0.000223,321.081522,2.423576,0.998384,0.006019,0.998521,0.009183,0.000551,0.001481,0.001480,0.000347,-0.000316,339.102410,2.405661,0.004242,52.0,2279.0,2.192308,0.999720,0.993659,18.900748,0.002647,21.0,957.0,2.000000,0.999720,0.997071,17.708995,0,0.004026
7,0-103,0.004120,1.000064,0.005331,1.000008,0.006557,0.000493,0.001040,0.001039,0.000133,-0.000246,324.608844,1.805239,1.002162,0.003397,1.002168,0.004459,0.000475,0.001139,0.001139,0.000188,-0.000187,300.954128,2.030422,0.002102,28.0,1181.0,2.071429,1.003108,0.997053,13.596726,0.001450,10.0,375.0,2.700000,1.003108,1.000589,5.887500,0,0.004023
8,0-109,0.002182,1.001546,0.001797,1.001583,0.003536,0.000202,0.000445,0.000445,0.000123,-0.000154,361.292373,2.474643,1.001976,0.001261,1.001972,0.002452,0.000212,0.000454,0.000454,0.000132,-0.000131,336.703390,4.619230,0.001266,45.0,1868.0,1.888889,1.002415,0.999928,17.775926,0.000679,20.0,759.0,1.750000,1.002415,1.001602,19.558333,0,0.004012
9,0-123,0.0026

## Model Building

In [ ]:
df_train.head()

,row_id,target,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id,stock_id_target_enc
0,0-5,0.004136,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0.002006,40.0,3179.0,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798,0,0.004025
1,0-11,0.001445,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0.000901,30.0,1289.0,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917,0,0.004057
2,0-16,0.002168,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0.001961,25.0,2161.0,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111,0,0.004050
3,0-31,0.002195,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0.001561,15.0,1962.0,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457,0,0.004025
4,0-62,0.001747,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0.000871,22.0,1791.0,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506,0,0.004025


In [ ]:
df_test.head()

,stock_id,time_id,row_id,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id_target_enc
0,0,4,0-4,1.000405,0.000294,1.00055,0.000252,0.000145,0.000557,0.000557,0.000393,-0.000115,350.666667,0.455667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,1.000344,1.000049,16.206349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028


## LightGBM

In [ ]:
import lightgbm as lgbm

In [ ]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [ ]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [ ]:
X

,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id,stock_id_target_enc
0,1.003725,0.004499,1.003661,0.006999,0.000388,0.000852,0.000852,0.000176,-0.000151,323.496689,10.392625,1.003753,0.002953,1.003667,0.004864,0.000372,0.000822,0.000822,0.000223,-0.000162,294.928058,13.547710,0.002006,40.0,3179.0,2.750000,1.004680,1.002301,23.118036,0.001308,21.0,1587.0,2.571429,1.004680,1.002561,18.628798,0,0.004025
1,1.000239,0.001204,1.000206,0.002476,0.000212,0.000394,0.000394,0.000142,-0.000135,411.450000,1.303403,1.000397,0.000981,1.000346,0.002009,0.000239,0.000353,0.000353,0.000164,-0.000123,484.521739,0.808444,0.000901,30.0,1289.0,1.900000,1.000828,0.999724,20.061111,0.000587,16.0,900.0,2.250000,1.000828,1.000072,26.822917,0,0.004057
2,0.999542,0.002369,0.999680,0.004801,0.000331,0.000725,0.000725,0.000197,-0.000198,416.351064,2.440987,0.998685,0.001295,0.998935,0.003196,0.000431,0.000689,0.000689,0.000141,-0.000249,455.235294,2.450092,0.001961,25.0,2161.0,2.720000,1.000455,0.997391,25.548476,0.001137,12.0,1189.0,3.166667,0.999928,0.997391,27.236111,0,0.004050
3,0.998832,0.002574,0.998633,0.003637,0.000380,0.000861,0.000860,0.000190,-0.000108,435.266667,2.360404,0.998436,0.001776,0.998455,0.002713,0.000331,0.000834,0.000833,0.000158,-0.000095,418.169811,1.909433,0.001561,15.0,1962.0,3.933333,1.000231,0.997972,33.507407,0.001089,9.0,1556.0,5.111111,0.999537,0.997972,34.623457,0,0.004025
4,0.999619,0.001894,0.999626,0.003257,0.000254,0.000397,0.000397,0.000191,-0.000109,343.221591,1.840490,0.999488,0.001520,0.999615,0.002188,0.000252,0.000425,0.000425,0.000191,-0.000120,407.584270,2.012114,0.000871,22.0,1791.0,4.045455,1.000023,0.999231,16.177243,0.000453,11.0,1219.0,4.909091,0.999749,0.999417,19.656506,0,0.004025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,0.999582,0.003691,0.999585,0.005876,0.000361,0.000878,0.000878,0.000091,-0.000202,406.045161,1.937377,0.999375,0.002899,0.999263,0.003776,0.000297,0.000890,0.000890,0.000091,-0.000162,379.443662,1.038553,0.002171,37.0,2570.0,2.783784,1.000522,0.998465,22.267192,0.001451,18.0,796.0,2.055556,0.999739,0.998563,19.007407,126,0.005286
428928,1.002476,0.004104,1.002602,0.004991,0.000295,0.000706,0.000706,0.000126,-0.000142,243.322870,1.178991,1.003528,0.003454,1.003709,0.003402,0.000280,0.000729,0.000729,0.000147,-0.000168,242.561905,1.497568,0.002180,43.0,2323.0,3.418605,1.005586,1.000620,11.801938,0.001791,20.0,1107.0,3.550000,1.005586,1.002005,10.883333,126,0.005322
428929,1.001082,0.003117,1.000996,0.006020,0.000394,0.000739,0.000739,0.000189,-0.000192,348.093750,0.828922,1.001282,0.002792,1.001195,0.005387,0.000430,0.000704,0.000704,0.000244,-0.000200,343.592814,0.424619,0.001921,35.0,3740.0,2.800000,1.001837,0.999605,37.524286,0.001580,24.0,2750.0,2.541667,1.001837,1.000395,45.681250,126,0.005322
428930,1.001809,0.003661,1.001790,0.005362,0.000231,0.000530,0.000530,0.000143,-0.000134,426.416040,5.392087,1.001807,0.002379,1.001774,0.003182,0.000199,0.000494,0.000493,0.000150,-0.000118,471.183246,4.676456,0.002051,80.0,9389.0,2.925000,1.002604,1.000889,36.360298,0.001520,43.0,5150.0,2.813953,1.002591,

In [ ]:
y

0         0.004136
1         0.001445
2         0.002168
3         0.002195
4         0.001747
            ...   
428927    0.003461
428928    0.003113
428929    0.004070
428930    0.003357
428931    0.002090
Name: target, Length: 428932, dtype: float64

In [ ]:
# functions for evaluation

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

In [ ]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 40,
      'learning_rate': 0.01, ##########
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

### Cross Validation

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=2046, shuffle=True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0                         # validation score

In [ ]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold :", fold+1)
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx] ######
    X_valid, y_valid = X.loc[val_idx], y[val_idx] ######
    
    #RMSPE weight
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight = weights) # weights???????

    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference = lgbm_train, weight = weights)
    
    # model 
    model = lgbm.train(params=params,
                      train_set=lgbm_train,
                      valid_sets=[lgbm_train, lgbm_valid],
                      num_boost_round=5000,         
                      feval=feval_RMSPE,
                      verbose_eval=100,
                      categorical_feature = ['stock_id']                
                     )
    
    # validation 
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred), 3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold : 1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.083714 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9545
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 38
[LightGBM] [Info] Start training from score 0.001800
Training until validation scores don't improve for 40 rounds
[100]	training's rmse: 0.000659712	training's RMSPE: 0.30534	valid_1's rmse: 0.000664086	valid_1's RMSPE: 0.30727
[200]	training's rmse: 0.000535599	training's RMSPE: 0.2479	valid_1's rmse: 0.000543316	valid_1's RMSPE: 0.25139
[300]	training's rmse: 0.000509671	training's RMSPE: 0.2359	valid_1's rmse: 0.000519399	valid_1's RMSPE: 0.24033
[400]	training's rmse: 0.000502007	training's RMSPE: 0.23235	valid_1's rmse: 0.000513934	valid_1's RMSPE: 0.2378
[500]	training's rmse: 0.000498023	training's RMSPE: 0.23051	valid_1's rmse: 0.000512732	valid_1's RMSPE: 0.23724
[600]	training's 

In [ ]:
scores

0.235

# Test set

In [ ]:
df_test.columns

Index(['stock_id', 'time_id', 'row_id', 'wap1_mean',
       'log_return1_realized_volatility', 'wap2_mean',
       'log_return2_realized_volatility', 'wap_balance_mean',
       'bid_ask_spread_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'total_volume_mean', 'volume_imbalance_mean',
       'wap1_mean_300', 'log_return1_realized_volatility_300', 'wap2_mean_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'bid_ask_spread_mean_300', 'price_spread_mean_300',
       'bid_spread_mean_300', 'ask_spread_mean_300', 'total_volume_mean_300',
       'volume_imbalance_mean_300', 'trade_log_return_realized_volatility',
       'trade_seconds_in_bucket_count_unique', 'trade_size_sum',
       'trade_order_count_mean', 'trade_price_max', 'trade_price_min',
       'trade_avg_size_mean', 'trade_log_return_realized_volatility_300',
       'trade_seconds_in_bucket_count_unique_300', 'trade_size_sum_300',
       'trade_order_count_mean_300', 'trad

In [ ]:
df_train.columns

Index(['row_id', 'target', 'wap1_mean', 'log_return1_realized_volatility',
       'wap2_mean', 'log_return2_realized_volatility', 'wap_balance_mean',
       'bid_ask_spread_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'total_volume_mean', 'volume_imbalance_mean',
       'wap1_mean_300', 'log_return1_realized_volatility_300', 'wap2_mean_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'bid_ask_spread_mean_300', 'price_spread_mean_300',
       'bid_spread_mean_300', 'ask_spread_mean_300', 'total_volume_mean_300',
       'volume_imbalance_mean_300', 'trade_log_return_realized_volatility',
       'trade_seconds_in_bucket_count_unique', 'trade_size_sum',
       'trade_order_count_mean', 'trade_price_max', 'trade_price_min',
       'trade_avg_size_mean', 'trade_log_return_realized_volatility_300',
       'trade_seconds_in_bucket_count_unique_300', 'trade_size_sum_300',
       'trade_order_count_mean_300', 'trade_price_max_300',
  

In [ ]:
y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis = 1)

In [ ]:
X_test

,stock_id,wap1_mean,log_return1_realized_volatility,wap2_mean,log_return2_realized_volatility,wap_balance_mean,bid_ask_spread_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_300,log_return1_realized_volatility_300,wap2_mean_300,log_return2_realized_volatility_300,wap_balance_mean_300,bid_ask_spread_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_price_max,trade_price_min,trade_avg_size_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_price_max_300,trade_price_min_300,trade_avg_size_mean_300,stock_id_target_enc
0,0,1.000405,0.000294,1.00055,0.000252,0.000145,0.000557,0.000557,0.000393,-0.000115,350.666667,0.455667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,1.000344,1.000049,16.206349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028


In [ ]:
target = np.zeros(len(X_test))

#light gbm models
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)

In [ ]:
y_pred = y_pred.assign(target = target)

In [ ]:
y_pred

,row_id,target
0,0-4,0.000920
1,0-32,0.000879
2,0-34,0.000879


In [ ]:
y_pred.to_csv('submission.csv',index = False)